In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [3]:
# url to API
url = 'https://ies-midterm.soulution.rocks/'

In [45]:
# post request to login to API
req = requests.post(url + "login", json={"cuni":"96640370"})
req_json = req.json()

{"data": {"personal_code": "f498a093", "dataset_ids": ["775352ba11", "e2beb850b2", "81395feb08", "40138ff3b8", "ef0bccd7cc"]}, "message": "Hello, Karl\u00ed\u010dek Ond\u0159ej! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook."}


In [47]:
# printing personal code
print("My personal code is: " + str(req_json['data']['personal_code']))

My personal code is: f498a093


In [49]:
# function for loading data
def load_data(dataset_id):
    r = requests.get(f"{url}data/{dataset_id}")
    return r.json()

In [95]:
# loading data
warning_message = 'Error [503] - Sorry, I just slipped and fell. Thank God, I am made of silicon. You need to work this out.'
datasets = []
for idd in req_json['data']['dataset_ids']:
    successful = False
    # loading the same dataset until it is successful
    while not successful:
        data = load_data(idd)
        if data['message'] == warning_message:
            # if the data weren't loaded continue the while
            continue
        else:
            # apend data to datasets list and break the while loop with successful
            datasets.append(data)
            successful = True

In [174]:
# creating pandas dataframes
df_all = pd.DataFrame(datasets[0]['data']['data'])
df_all = df_all.melt(id_vars=['Date'], var_name="type", value_name=datasets[0]['data']['company']).set_index(['Date', 'type'])
df_all[datasets[0]['data']['company']] = pd.to_numeric(df_all[datasets[0]['data']['company']], downcast="float")

for data in datasets[1:]:
    df = pd.DataFrame(data['data']['data'])
    df = df.melt(id_vars=['Date'], var_name="type", value_name=data['data']['company']).set_index(['Date', 'type'])
    df[data['data']['company']] = pd.to_numeric(df[data['data']['company']], downcast="float")
    
    df_all = df_all.join(df, how="outer")

df_all.tail(n=20)

SMIN         IAG           CCH            EVR  \
Date       type                                                               
2020-11-17 Open          37.410000        3.47  1.892000e+01      87.879997   
           Volume     16600.000000  6726700.00  4.918720e+05  309400.000000   
2020-11-18 Adj.Close     37.599998        3.36  1.882000e+01      88.309998   
           Close         37.599998        3.36  1.882000e+01      88.309998   
           High          37.900002        3.45  1.924000e+01      90.690002   
           Low           37.590000        3.34  1.874000e+01      88.279999   
           Open          37.900002        3.45  1.895000e+01      89.180000   
           Volume     31500.000000  5219400.00  7.567850e+05  334200.000000   
2020-11-19 Adj.Close     37.970001        3.37  1.910000e+01      88.489998   
           Close         37.970001        3.37  1.910000e+01      88.489998   
           High          38.000000        3.41  1.950000e+01      88.550003   
           Low           37.790001        3.31  1.865000e+01      87.250000   
           Open          37.790001        3.32  1.885000e+01      87.820000   
           Volume     18100.000000  5621800.00  7.251840e+05  166300.000000   
2020-11-20 Adj.Close     38.169998        3.41  1.939000e+01      88.309998   
           Close         38.169998        3.41  1.939000e+01      88.309998   
           High          38.240002        3.51  1.950000e+01      88.570000   
           Low           38.169998        3.40  1.907000e+01      87.160004   
           Open          38.189999        3.42  1.929000e+01      87.980003   
           Volume     10500.000000  6329600.00  1.020227e+06  196500.000000   

                      CRDA  
Date       type             
2020-11-17 Open        NaN  
           Volume      NaN  
2020-11-18 Adj.Close   NaN  
           Close       NaN  
           High        NaN  
           Low         NaN  
           Open        NaN  
           Volume      NaN  
2020-11-19 Adj.Close   NaN  
           Close       NaN  
           High        NaN  
           Low         NaN  
           Open        NaN  
           Volume      NaN  
2020-11-20 Adj.Close   NaN  
           Close       NaN  
           High        NaN  
           Low         NaN  
           Open        NaN  
           Volume      NaN

In [ ]:
# Part 2

In [175]:
df_all.reset_index(inplace=True)

In [176]:
df_all = df_all[(df_all['Date'] >= "2019-01-02") & (df_all['Date'] <= "2020-11-20")]
df_all.head()

,Date,type,SMIN,IAG,CCH,EVR,CRDA
6,2019-01-02,Adj.Close,37.300087,3.69,9.623075,69.908577,NaN
7,2019-01-02,Close,38.490002,3.69,9.650000,73.839996,NaN
8,2019-01-02,High,38.560001,3.76,9.650000,74.230003,NaN
9,2019-01-02,Low,38.330002,3.60,9.640000,69.980003,NaN
10,2019-01-02,Open,38.389999,3.68,9.650000,70.110001,NaN


In [177]:
# do we have data for all period
df_all[df_all['type'] == 'Adj.Close'].isna().sum()[2:]
# we can see that we have 476 missing days for CRDA

SMIN      0
IAG       0
CCH       0
EVR       0
CRDA    476
dtype: int64

In [178]:
# highest and lowest adj. close
df_all[df_all['type'] == 'Adj.Close'].drop(columns=['Date','type']).aggregate([np.min, np.max], axis = 0)
# first row is min for stock and second row is maximum

,SMIN,IAG,CCH,EVR,CRDA
amin,20.771763,1.88,9.603131,35.080837,5136.0
amax,39.713108,5.20,19.389999,93.306076,5153.0


In [186]:
# find monthly returns. Which company has the highest recorded? 
# month corresponds approximately to 21 trading days
df_adjclose = df_all[df_all['type'] == 'Adj.Close'].drop(columns=['Date','type'])

df_returns = df_adjclose.pct_change(periods=21)
df_returns.max()[df_returns.max() == df_returns.max().max()]

IAG    0.62766
dtype: float32

In [191]:
# 10 day std
df_adjclose.drop(columns=['CRDA']).rolling(10).std().tail()

,SMIN,IAG,CCH,EVR
2844,0.613270,0.173333,0.695142,2.765931
2850,0.514373,0.178076,0.743374,2.510661
2856,0.568960,0.191183,0.790960,2.326013
2862,0.655233,0.140238,0.867910,2.223760
2868,0.744163,0.075285,0.910748,1.435681
